<span style="font-size: 20pt; font-weight: bold; color: #0098cd;">Named-Entity Recognition</span>

# Parte 1: carga y preprocesamiento del texto a analizar

In [1]:
import pathlib
import spacy
import pandas as pd
from spacy import displacy
import csv
import es_core_news_md

/Users/davidcamilo/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


El siguiente código simplemente carga y preprocesa el texto. Para ello, lo primero que hace es cargar un modelo de lenguaje previamente entrenado. En este caso, se utiliza <i>es_core_news_md</i>: 

https://spacy.io/models/es#es_core_news_md


In [2]:
nlp = es_core_news_md.load()

El objeto <i>nlp</i> permite utilizar el modelo de lenguaje cargado, de forma que se puede procesar un texto y obtenerlo en su versión preprocesada. Así, nos permite realizar las diferentes tareas. En este caso, vamos a utilizar el pipeline para hacer un preprocesamiento básico, que consiste en tokenizar el texto.

In [3]:
filename = "./02Dataset_sin_procesar.csv"
lines_number = 574915
#lines_number = 30000
data = pd.read_csv(filename, delimiter=';')  
#data = pd.read_csv(filename, delimiter=';',nrows=lines_number)  

/var/folders/nb/q8sclyy53zj0glnln2j4qry00000gn/T/ipykernel_9256/400137604.py:4: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filename, delimiter=';')


El código anterior carga el archivo CSV (opcionalmente con un límite de líneas a leer) y genera la variable <i>data</i>, que contiene un Dataframe (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) con los datos leídos del CSV.

In [4]:
# Lista para guardar los índices válidos
valid_indices = []
excluded_count = 0

for idx, val in enumerate(data["INTENSIDAD"]):
    try:
        # Intentar convertir a número (float o int)
        numeric_val = float(val)
        
        # Verificar si es un entero (como 3.0) y no algo como 3.5
        if numeric_val.is_integer():
            valid_indices.append(idx)
        else:
            excluded_count += 1
    except:
        excluded_count += 1

print(f"Se excluyeron {excluded_count} entradas debido a que 'INTENSIDAD' no es un valor numérico entero.")

doc = []
value = []

for i in valid_indices:
    if i % 10000 == 0 and i != 0:
        print(f"Procesadas {i} líneas")

    tmp_doc = nlp(data["CONTENIDO A ANALIZAR"][i])
    doc.append(tmp_doc)

    tmp_value = int(float(data["INTENSIDAD"][i]))  # Convertimos seguro a int
    value.append(tmp_value)


Se excluyeron 276 entradas debido a que 'INTENSIDAD' no es un valor numérico entero.
Procesadas 10000 líneas
Procesadas 20000 líneas
Procesadas 30000 líneas
Procesadas 40000 líneas
Procesadas 50000 líneas
Procesadas 60000 líneas
Procesadas 70000 líneas
Procesadas 80000 líneas
Procesadas 90000 líneas
Procesadas 100000 líneas
Procesadas 110000 líneas
Procesadas 120000 líneas
Procesadas 130000 líneas
Procesadas 140000 líneas
Procesadas 150000 líneas
Procesadas 160000 líneas
Procesadas 170000 líneas
Procesadas 180000 líneas
Procesadas 190000 líneas
Procesadas 200000 líneas
Procesadas 210000 líneas
Procesadas 220000 líneas
Procesadas 230000 líneas
Procesadas 240000 líneas
Procesadas 250000 líneas
Procesadas 260000 líneas
Procesadas 270000 líneas
Procesadas 280000 líneas
Procesadas 290000 líneas
Procesadas 300000 líneas
Procesadas 310000 líneas
Procesadas 320000 líneas
Procesadas 330000 líneas
Procesadas 340000 líneas
Procesadas 350000 líneas
Procesadas 360000 líneas
Procesadas 370000 líneas

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">¿Cuántos registros contiene el corpus?</span>

In [5]:
print(f"Total de registros antes del filtrado: {len(data)}")
print(f"Total de registros válidos (INTENSIDAD numérica entera): {len(valid_indices)}")
print(f"Total excluidos: {len(data) - len(valid_indices)}")


Total de registros antes del filtrado: 574915
Total de registros válidos (INTENSIDAD numérica entera): 574639
Total excluidos: 276


<b>Se usó len(data) para contar los registros originales en el CSV, dando un total de 574.915 entradas. Sin embargo, para el análisis posterior se excluyeron 276 registros porque el campo INTENSIDAD no era un número entero. Esto representa apenas el 0.05% del total, por lo que no afecta significativamente el análisis. El corpus final quedó con 574.639 registros válidos.</b>
<hr>
 

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">¿Cuántas palabras totales hay en los comentarios del corpus?</span>

In [6]:
total_palabras = sum(len([token for token in comentario if token.is_alpha]) for comentario in doc)
print(f"El corpus contiene un total de {total_palabras} palabras.")

El corpus contiene un total de 60439308 palabras.


<b>Se calculó la cantidad de palabras al contar solo los tokens alfabéticos de cada comentario. Esto excluye signos de puntuación y números, lo cual es adecuado para análisis semántico. El resultado fue de 60.439.308 palabras</b>
<hr>
 

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">¿Cuál el número promedio de palabras en cada comentario?</span>

In [7]:
promedio_palabras = total_palabras / len(doc)
print(f"El número promedio de palabras por comentario es {promedio_palabras:.2f}")

El número promedio de palabras por comentario es 105.18


<b>Se dividió el total de palabras por el número de comentarios procesados, arrojando un promedio de 105,18 palabras por comentario. Esto indica que la mayoría de mensajes son bastante extensos.</b>
<hr>
 

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Considerando dos grupos de comentarios (odio y no odio) ¿Cuál el número promedio de palabras en los comentarios de cada grupo?</span>

In [8]:
# Separar grupos usando INTENSIDAD como criterio binario
grupo_odio = [doc[i] for i in range(len(doc)) if value[i] >= 1]
grupo_no_odio = [doc[i] for i in range(len(doc)) if value[i] == 0]

# Función auxiliar
def contar_palabras(comentarios):
    return [len([token for token in c if token.is_alpha]) for c in comentarios]

# Cálculos
porcentaje_comentarios_odio = (len(grupo_odio) / len(doc)) * 100
porcentaje_comentarios_no_odio = (len(grupo_no_odio) / len(doc)) * 100

promedio_palabras_odio = sum(contar_palabras(grupo_odio)) / len(grupo_odio)
promedio_palabras_no_odio = sum(contar_palabras(grupo_no_odio)) / len(grupo_no_odio)

print(f"Cantidad de comentarios con odio: {len(grupo_odio)} ({porcentaje_comentarios_odio:.2f}%)")
print(f"Cantidad de comentarios sin odio: {len(grupo_no_odio)} ({porcentaje_comentarios_no_odio:.2f}%)")

print(f"Promedio de palabras en comentarios con odio: {promedio_palabras_odio:.2f}")
print(f"Promedio de palabras en comentarios sin odio: {promedio_palabras_no_odio:.2f}")


Cantidad de comentarios con odio: 12296 (2.14%)
Cantidad de comentarios sin odio: 562343 (97.86%)
Promedio de palabras en comentarios con odio: 15.60
Promedio de palabras en comentarios sin odio: 107.14


<b>Separé los comentarios en dos grupos según la intensidad: cuando esta es cero, se clasifican como comentarios sin odio (97.86%); en caso contrario, contienen odio (2.14%). El promedio de palabras en los comentarios con odio es de 15.6, mientras que en los comentarios sin odio asciende a 107.14. La diferencia es notable. Esto sugiere que los mensajes de odio tienden a ser más directos, breves y agresivos. Esta métrica, por sí sola, podría servir como un fuerte indicador para la clasificación en tareas de detección de discurso de odio.</b>
<hr>
 

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">¿Cuál es el número promedio de oraciones en los comentarios de cada grupo?</span>

In [9]:
def contar_oraciones(comentarios):
    return [len(list(c.sents)) for c in comentarios]

promedio_sents_odio = sum(contar_oraciones(grupo_odio)) / len(grupo_odio)
promedio_sents_no_odio = sum(contar_oraciones(grupo_no_odio)) / len(grupo_no_odio)

print(f"Promedio de oraciones (ODIO): {promedio_sents_odio:.2f}")
print(f"Promedio de oraciones (NO ODIO): {promedio_sents_no_odio:.2f}")# Incluye aquí el código generado para poder responder a tu pregunta


Promedio de oraciones (ODIO): 1.55
Promedio de oraciones (NO ODIO): 3.99


<b>Los mensajes con odio contienen en promedio 1.55 oraciones, mientras que los mensajes sin odio tienen 3.99. Esto refuerza lo observado anteriormente: los mensajes con odio son más breves, probablemente más emocionales y con menos desarrollo argumentativo.</b>
<hr>
 

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">¿Cuál es el porcentaje de comentarios que contienen entidades NER en cada grupo?</span>

In [10]:
def tiene_entidades(comentarios):
    return sum(1 for c in comentarios if len(c.ents) > 0)

porc_ner_odio = tiene_entidades(grupo_odio) / len(grupo_odio) * 100
porc_ner_no_odio = tiene_entidades(grupo_no_odio) / len(grupo_no_odio) * 100

print(f"% de comentarios con entidades NER (ODIO): {porc_ner_odio:.2f}%")
print(f"% de comentarios con entidades NER (NO ODIO): {porc_ner_no_odio:.2f}%")


% de comentarios con entidades NER (ODIO): 36.52%
% de comentarios con entidades NER (NO ODIO): 59.38%


<b>Solo el 36.52% de los mensajes con odio contienen entidades NER, frente al 59.38% de los que no contienen odio. Esto podría indicar que los comentarios de odio tienden a ser más generales, despersonalizados o coloquiales, mientras que los mensajes informativos o constructivos hacen referencia a personas, lugares o instituciones.</b>
<hr>


<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">¿Cuál es el porcentaje de comentarios que contienen entidades NER de tipo PERSON en cada grupo?</span>

In [11]:
def tiene_person(comentarios):
    return sum(1 for c in comentarios if any(ent.label_ == "PER" or ent.label_ == "PERSON" for ent in c.ents))

porc_person_odio = tiene_person(grupo_odio) / len(grupo_odio) * 100
porc_person_no_odio = tiene_person(grupo_no_odio) / len(grupo_no_odio) * 100

print(f"% de comentarios con entidad PERSON (ODIO): {porc_person_odio:.2f}%")
print(f"% de comentarios con entidad PERSON (NO ODIO): {porc_person_no_odio:.2f}%")


% de comentarios con entidad PERSON (ODIO): 17.59%
% de comentarios con entidad PERSON (NO ODIO): 28.77%


<b>Solo el 17.59% de los comentarios con odio mencionan entidades de tipo PERSON, en comparación con el 28.77% en los comentarios sin odio. Esto sugiere que los mensajes de odio, en su mayoría, no están dirigidos a personas específicas, sino que podrían estar enfocados en grupos, ideas, identidades colectivas o instituciones.</b>
<hr>
 

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Considerando dos grupos de comentarios (odio y no odio) ¿Cuál es el porcentaje de palabras en cada combinación posible de género y número (p.ej. masculino singular) en cada grupo?</span>

In [12]:
from collections import Counter

# Función para obtener combinaciones género+número por palabra
def extraer_genero_numero(comentarios):
    combinaciones = []
    for doc in comentarios:
        for token in doc:
            if token.pos_ in ["NOUN", "ADJ", "PROPN"]:
                genero = token.morph.get("Gender")
                numero = token.morph.get("Number")
                if genero and numero:
                    combinaciones.append(f"{genero[0]}_{numero[0]}")
    return Counter(combinaciones)

# Recuento en cada grupo
gn_odio = extraer_genero_numero(grupo_odio)
gn_no_odio = extraer_genero_numero(grupo_no_odio)

# Totales
total_gn_odio = sum(gn_odio.values())
total_gn_no_odio = sum(gn_no_odio.values())

# Unir todas las claves posibles
todas_las_claves = set(gn_odio.keys()) | set(gn_no_odio.keys())

# Imprimir tabla
print(f"{'Género_Número':<15} {'% Odio':>10} {'% No Odio':>12}")
print("-" * 40)

for clave in sorted(todas_las_claves):
    porcentaje_odio = (gn_odio[clave] / total_gn_odio * 100) if clave in gn_odio else 0
    porcentaje_no_odio = (gn_no_odio[clave] / total_gn_no_odio * 100) if clave in gn_no_odio else 0
    print(f"{clave:<15} {porcentaje_odio:>9.2f}% {porcentaje_no_odio:>11.2f}%")


Género_Número       % Odio    % No Odio
----------------------------------------
Fem_Plur             9.26%       12.28%
Fem_Sing            35.98%       35.43%
Masc_Plur           18.42%       16.43%
Masc_Sing           36.34%       35.85%


<b>Los resultados muestran que en ambos grupos predominan las formas masculino singular y femenino singular (alrededor del 36% cada una). No obstante, las diferencias más interesantes aparecen en los plurales: en los comentarios con odio, el masculino plural (18.42%) casi duplica al femenino plural (9.26%), mientras que en los comentarios sin odio esta brecha es mucho menor (16.43% vs. 12.28%). Esto sugiere una mayor presencia de formas masculinas en discursos de odio, especialmente al referirse a colectivos.</b>
<hr>
 

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Considerando dos grupos de comentarios (odio y no odio), indica cuántas entidades de cada tipo posible se reconocen en cada uno de los grupos</span>

In [13]:
def contar_entidades_por_tipo(comentarios):
    contador = Counter()
    for doc in comentarios:
        for ent in doc.ents:
            contador[ent.label_] += 1
    return contador

# Conteo por grupo
ner_odio = contar_entidades_por_tipo(grupo_odio)
ner_no_odio = contar_entidades_por_tipo(grupo_no_odio)

# Totales por grupo
total_odio = sum(ner_odio.values())
total_no_odio = sum(ner_no_odio.values())

# Unir todos los tipos de entidades detectados
todos_los_tipos = set(ner_odio.keys()) | set(ner_no_odio.keys())

# Imprimir en forma de tabla con número y porcentaje
print(f"{'Tipo de Entidad':<20} {'Con Odio':>20} {'Sin Odio':>20}")
print("-" * 62)

for tipo in sorted(todos_los_tipos):
    cant_odio = ner_odio.get(tipo, 0)
    cant_no_odio = ner_no_odio.get(tipo, 0)

    porc_odio = (cant_odio / total_odio * 100) if total_odio else 0
    porc_no_odio = (cant_no_odio / total_no_odio * 100) if total_no_odio else 0

    col_odio = f"{cant_odio} ({porc_odio:.2f}%)"
    col_no_odio = f"{cant_no_odio} ({porc_no_odio:.2f}%)"

    print(f"{tipo:<20} {col_odio:>20} {col_no_odio:>20}")


Tipo de Entidad                  Con Odio             Sin Odio
--------------------------------------------------------------
LOC                         1596 (25.50%)      742772 (40.30%)
MISC                        1136 (18.15%)      207269 (11.25%)
ORG                          933 (14.91%)      220097 (11.94%)
PER                         2593 (41.43%)      673053 (36.52%)


<b>En los comentarios sin odio, el contenido se concentra principalmente en dos categorías: LOC (40.30%) y PERSON (36.52%). Esto sugiere que los mensajes no hostiles tienden a mencionar lugares y personas de forma predominante, con poca presencia de otras entidades.

Por el contrario, en los comentarios con odio, aunque PERSON sigue siendo la categoría dominante (41.43%), hay una distribución más equilibrada entre otras entidades. LOC (25.50%) y MISC (18.15%) también tienen un peso relevante, lo que indica que el discurso de odio, si bien se enfoca mayormente en personas, también involucra referencias significativas a lugares e identidades diversas.</b>
<hr>
 

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">Considerando dos grupos de comentarios (odio y no odio), extrae y muestra los 100 lemas más repetidos en los comentarios de cada grupo</span>

In [14]:
def contar_lemas(comentarios):
    lemas = [token.lemma_.lower() for doc in comentarios for token in doc if token.is_alpha and not token.is_stop]
    return Counter(lemas).most_common(100)

print(">> 100 lemas más comunes (ODIO):")
for lemma, count in contar_lemas(grupo_odio):
    print(f"{lemma}: {count}")

print("\n>> 100 lemas más comunes (NO ODIO):")
for lemma, count in contar_lemas(grupo_no_odio):
    print(f"{lemma}: {count}")


>> 100 lemas más comunes (ODIO):
mierda: 913
puta: 776
gobierno: 542
asco: 529
panfleto: 489
q: 461
país: 459
hijo: 449
españa: 432
mentiroso: 426
gente: 395
vergüenza: 373
basura: 349
gentuza: 338
español: 316
tonto: 290
político: 274
dejar: 261
pasar: 256
miserable: 254
sinvergüenza: 243
idiota: 234
ver: 233
pagar: 229
culo: 226
menudo: 221
año: 218
mundo: 218
gilipol él: 216
puto: 215
tener: 214
salir: 211
inútil: 210
dinero: 209
decir: 206
dar: 204
haber: 202
madre: 201
vida: 199
seguir: 195
terrorista: 195
facha: 191
querer: 190
fascista: 188
informativo: 181
ir: 180
poner: 180
noticia: 179
madrid: 175
mujer: 173
tomar: 170
cárcel: 165
creer: 165
comunista: 162
persona: 161
hdp: 160
cara: 157
hdlgp: 157
payaso: 154
esperar: 154
quedar: 153
izquierda: 152
asesino: 151
panda: 151
importar: 149
vivir: 148
mentira: 147
cosa: 145
sánchez: 144
deber: 144
asqueroso: 143
vacuna: 138
jeta: 137
trump: 136
pensar: 135
votar: 135
partido: 134
hablar: 133
terrorismo: 132
venir: 131
maldito: 13

<b>En los comentarios con odio, los lemas más frecuentes incluyen términos claramente ofensivos y peyorativos como "mierda", "puta", "asco", "basura", "gentuza", "idiota", "inútil", y otros insultos directos. También aparecen con alta frecuencia palabras relacionadas con política o nacionalidad como "gobierno", "españa", "español", "sánchez", "comunista", "fascista", "vox", lo que sugiere que muchos mensajes de odio están ligados a temas ideológicos o de identidad nacional.

En contraste, en los comentarios sin odio predominan lemas mucho más neutros o informativos, como "año", "gobierno", "persona", "caso", "mujer", "vacuna", "trabajo", "vida", "familia" y "salud". Esto refuerza la idea de que los comentarios no agresivos tienden a referirse a hechos, contextos sociales y situaciones cotidianas, con un lenguaje más formal y descriptivo.

Además, mientras que en los mensajes sin odio predominan sustantivos concretos y verbos relacionados con acciones o estados (como "tener", "haber", "poder", "esperar", "vivir"), en los de odio hay mayor presencia de insultos, adjetivos calificativos intensos y términos despectivos.</b>
<hr>
 

<span style="font-size: 14pt; font-weight: bold; color: #0098cd;">¿Es posible utilizar alguna de las características extraídas en las preguntas anteriores para determinar si un mensaje contiene odio? Justifica tu respuesta con el análisis estadístico que consideres necesario.</span>

In [32]:
# --- 3. Construcción del DataFrame con todas las características
rows = []

for i in range(len(doc)):
    comentario = doc[i]
    intensidad_i = value[i]
    
    # Palabras y oraciones
    palabras = [t for t in comentario if t.is_alpha]
    num_palabras = len(palabras)
    num_oraciones = len(list(comentario.sents))
    
    # Entidades
    tiene_ner = 1 if len(comentario.ents) > 0 else 0
    tiene_person = 1 if any(ent.label_ in ["PER", "PERSON"] for ent in comentario.ents) else 0
    
    # Género y número
    genero_num = contar_genero_numero(comentario)
    
    # Entidades NER por tipo
    entidades = contar_entidades_por_tipo(comentario)
    
    # Lemas tóxicos
    count_toxicos, ratio_toxicos = contar_lemas_toxicos(comentario)
    
    row = {
        'intensidad': intensidad_i,
        'num_palabras': num_palabras,
        'num_oraciones': num_oraciones,
        'tiene_ner': tiene_ner,
        'tiene_person': tiene_person,
        # Género y número (ratios)
        'ratio_fem_sing': genero_num['fem_sing'] / num_palabras if num_palabras else 0,
        'ratio_fem_plur': genero_num['fem_plur'] / num_palabras if num_palabras else 0,
        'ratio_masc_sing': genero_num['masc_sing'] / num_palabras if num_palabras else 0,
        'ratio_masc_plur': genero_num['masc_plur'] / num_palabras if num_palabras else 0,
        # Entidades por tipo (ratios)
        'ratio_LOC': entidades['LOC'] / num_palabras if num_palabras else 0,
        'ratio_MISC': entidades['MISC'] / num_palabras if num_palabras else 0,
        'ratio_ORG': entidades['ORG'] / num_palabras if num_palabras else 0,
        'ratio_PER': entidades['PER'] / num_palabras if num_palabras else 0,
        # Lemas tóxicos
        'count_lemas_toxicos': count_toxicos,
        'ratio_lemas_toxicos': ratio_toxicos
    }
    rows.append(row)

# Convertimos a DataFrame
df_features = pd.DataFrame(rows)
tabla_ratios_por_intensidad = df_features.groupby('intensidad').mean(numeric_only=True).T
tabla_ratios_por_intensidad

intensidad,0,1,2,3,4,5,6
num_palabras,107.14,16.44,32.53,31.24,14.95,16.76,26.00
num_oraciones,3.99,1.53,1.90,2.22,1.54,1.42,1.85
tiene_ner,0.59,0.31,0.44,0.48,0.37,0.35,0.31
tiene_person,0.29,0.12,0.13,0.19,0.18,0.18,0.15
ratio_fem_sing,0.10,0.08,0.08,0.08,0.12,0.09,0.07
ratio_fem_plur,0.03,0.03,0.02,0.02,0.02,0.02,0.03
ratio_masc_sing,0.10,0.18,0.09,0.07,0.11,0.09,0.06
ratio_masc_plur,0.04,0.07,0.06,0.05,0.06,0.03,0.04
ratio_LOC,0.02,0.01,0.03,0.02,0.01,0.00,0.00
ratio_MISC,0.02,0.01,0.01,0.01,0.02,0.01,0.00


<b>Sí, es posible utilizar varias de las características extraídas para determinar si un mensaje contiene odio. Aunque el corpus está altamente desbalanceado (solo el 2.14% de los mensajes tienen alguna intensidad de odio), el análisis por niveles de intensidad permite observar patrones claros y consistentes que justifican su uso como variables predictivas.

A partir de la tabla agregada por niveles de intensidad, se pueden destacar los siguientes hallazgos:

Los mensajes con mayor intensidad de odio son significativamente más cortos, tanto en número de palabras como en número de oraciones. Por ejemplo, los comentarios sin odio tienen en promedio 107 palabras y 3.99 oraciones, mientras que los de intensidad 1 tienen apenas 16 palabras y 1.53 oraciones.

La presencia de entidades NER y especialmente de tipo PERSON disminuye claramente en los mensajes con odio. Esto sugiere que estos comentarios tienden a ser menos informativos y más generalistas o emocionales.

Aunque el número absoluto de lemas tóxicos en comentarios con odio no es alto, la proporción de lemas tóxicos sobre el total de palabras es mucho mayor. En mensajes sin odio, el ratio es de 0.13, mientras que en mensajes con intensidad 1 sube hasta 0.31, lo que implica un lenguaje más agresivo y ofensivo.

Las variables morfológicas de género y número (como ratio_masc_sing o ratio_fem_sing) muestran ligeras variaciones, pero no presentan una tendencia clara que permita usarlas como predictores principales.

En conjunto, este análisis muestra que variables como; número de palabras, número de oraciones, presencia de entidades NER y PERSON, y proporción de lemas tóxicos tienen un comportamiento diferencial consistente y pueden utilizarse como señales para construir un modelo de detección de odio. Aunque no son perfectas de forma individual, combinadas tienen valor discriminativo y justifican su inclusión en modelos más complejos, especialmente si se complementan con técnicas para tratar el desbalance del corpus.</b>
<hr>
 